In [1]:
import cv2
import torch
import numpy as np
import insightface
from insightface.app import FaceAnalysis
from insightface.model_zoo import get_model
from ultralytics import YOLO
import os
import pickle
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances

In [2]:
yolo_model = YOLO("yolov11s-face.pt")

In [3]:
app = FaceAnalysis(name="buffalo_l", providers=['CUDAExecutionProvider'])
app.prepare(ctx_id=0)

Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with options: {'CUDAExecutionProvider': {'device_id': '0', 'has_user_compute_stream': '0', 'cudnn_conv1d_pad_to_nc1d': '0', 'user_compute_stream': '0', 'gpu_external_alloc': '0', 'gpu_mem_limit': '18446744073709551615', 'enable_cuda_graph': '0', 'gpu_external_free': '0', 'gpu_external_empty_cache': '0', 'arena_extend_strategy': 'kNextPowerOfTwo', 'cudnn_conv_algo_search': 'EXHAUSTIVE', 'do_copy_in_default_stream': '1', 'cudnn_conv_use_max_workspace': '1', 'tunable_op_enable': '0', 'tunable_op_tuning_enable': '0', 'tunable_op_max_tuning_duration_ms': '0', 'enable_skip_layer_norm_strict_mode': '0', 'prefer_nhwc': '0', 'use_ep_level_unified_stream': '0', 'use_tf32': '1', 'sdpa_kernel': '0', 'fuse_conv_bias': '0'}, 'CPUExecutionProvider': {}}
find model: C:\Users\Admin/.insightface\models\buffalo_l\1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'

In [4]:
import torch
import onnxruntime as ort

print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)
print("cuDNN version:", torch.backends.cudnn.version())
print("GPU name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU detected")

# Kiểm tra phiên bản ONNX Runtime
print("ONNX Runtime version:", ort.__version__)
print("ONNX Runtime available providers:", ort.get_available_providers())


PyTorch version: 2.5.1+cu121
CUDA available: True
CUDA version: 12.1
cuDNN version: 90100
GPU name: NVIDIA GeForce GTX 1650
ONNX Runtime version: 1.20.0
ONNX Runtime available providers: ['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider']


In [5]:
img = cv2.imread("data\Quang\IMG_4688.JPG")
result = yolo_model.predict(img)


0: 640x448 1 face, 45.0ms
Speed: 7.5ms preprocess, 45.0ms inference, 5.5ms postprocess per image at shape (1, 3, 640, 448)


In [6]:
for box in result[0].boxes.xyxy:
    x1, y1, x2, y2 = map(int, box[:4])

    margin_x = int((x2 - x1) * 0.2)  # 20% of the width
    margin_y = int((y2 - y1) * 0.2)  # 20% of the height

    # Adjust the coordinates with the margin
    x1 = max(0, x1 - margin_x)
    y1 = max(0, y1 - margin_y)
    x2 = min(img.shape[1], x2 + margin_x)
    y2 = min(img.shape[0], y2 + margin_y)
    
    face_crop = img[y1:y2, x1:x2]
    face_crop_rgb = cv2.cvtColor(face_crop, cv2.COLOR_BGR2RGB)
    # face_crop_rgb = cv2.resize(face_crop, (112, 112))
    face_crop_rgb = face_crop_rgb.astype(np.uint8)
    cv2.imwrite("face_crop.jpg", face_crop_rgb)

In [7]:
face_crop_rgb.shape

(484, 326, 3)

In [8]:
img_crop = cv2.imread("face_crop.jpg")
img_crop = cv2.cvtColor(img_crop, cv2.COLOR_BGR2RGB)

In [9]:
emb= app.get(face_crop_rgb)

c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\insightface\utils\transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4


In [10]:
emb

[{'bbox': array([     48.391,      62.582,      281.22,      412.02], dtype=float32),
  'kps': array([[     105.83,      203.68],
         [     215.94,      201.85],
         [     159.61,      269.51],
         [      120.8,       332.5],
         [     203.17,       331.6]], dtype=float32),
  'det_score': 0.75946206,
  'landmark_3d_68': array([[     38.447,      204.05,      167.62],
         [     44.608,      240.28,      163.88],
         [     52.026,       276.9,      157.99],
         [     59.805,      312.15,      147.16],
         [     69.936,      345.18,      122.76],
         [     83.497,      370.62,       100.1],
         [     100.71,      389.86,      80.691],
         [     123.09,       403.7,      56.638],
         [     159.87,      413.33,      44.018],
         [     197.59,      403.23,      54.647],
         [     226.64,      386.12,       79.32],
         [     247.66,      361.21,      103.23],
         [     260.49,      338.13,      126.51],
         [

In [11]:
emb[0].embedding

array([    0.19382,      1.7445,    -0.51876,      2.5214,    -0.72321,    -0.26674,      1.6255,     -2.8853,    -0.31862,     -1.0088,    -0.90917,      0.3593,     0.27167,     -1.8938,     0.87729,      2.8866,      1.6875,      1.3909,     0.90229,    -0.44753,    -0.10557,      0.4107,    -0.78817,       1.056,
           -2.8566,     0.86741,     0.85019,     -1.5084,     0.76658,      2.1931,      1.0382,     0.53307,    0.069566,     0.68498,    -0.44198,     0.81637,     -1.5428,     0.47405,    -0.86779,      1.2113,     -0.4948,   -0.049193,       1.426,    0.093436,    0.014479,      1.1729,    0.099357,      1.4749,
            1.1602,      2.3267,      1.1687,     0.73367,     0.50468,     0.55418,     0.10221,      1.1905,    -0.45278,     -1.4751,       1.249,     -1.2594,    -0.78967,    -0.31666,    -0.33283,     0.82492,     0.70559,    -0.87452,      2.0286,  -0.0087101,     -1.9152,     -1.8469,     -1.3434,     0.40987,
          -0.58682,      0.5654,     0.1571

In [12]:
emb[0].embedding.shape

(512,)

In [1]:
import pickle

with open("data/features/face_embeddings.pkl", "rb") as f:
    embeddings = pickle.load(f)

# In toàn bộ nội dung (có thể nhiều)
print(embeddings)

{'Phuc': [[0.22303201258182526, -1.0530744791030884, 1.6669842004776, 0.6542799472808838, -0.8480203747749329, -1.3123224973678589, 0.6609347462654114, 0.12105949223041534, 0.14378421008586884, 0.9741726517677307, -0.38707247376441956, 0.6305508613586426, -1.5540801286697388, -1.6121097803115845, 0.6126217246055603, 1.9063185453414917, 0.7874377369880676, 1.7334612607955933, 0.8715962767601013, -0.9689580202102661, -0.26685038208961487, 0.01835416443645954, -0.9175182580947876, 2.0208587646484375, -0.23742049932479858, -0.07125353813171387, -0.9010850191116333, -0.1584712713956833, -0.2859993278980255, 0.7903450727462769, 0.47700875997543335, -0.20233193039894104, -0.642142117023468, -0.5010821223258972, 1.8739947080612183, -1.3769338130950928, 0.46534618735313416, 0.5435901880264282, 1.1574621200561523, 0.17373156547546387, 1.875093698501587, 0.6240994334220886, 0.6220753788948059, 0.8972064852714539, 1.3532108068466187, 1.744397521018982, 1.3871866464614868, 1.3290561437606812, 1.530

In [2]:
import pickle

with open("data/features/face_embeddings.pkl", "rb") as f:
    embeddings = pickle.load(f)

# In toàn bộ tên (key) trong file .pkl
print("Danh sách person_name có trong file:")
for person_name in embeddings.keys():
    print("-", person_name)

Danh sách person_name có trong file:
- Phuc
- Quang
